In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [18]:
training_dataset_path = "./monkey_images/training"
test_dataset_path = "./monkey_images/validation"

In [19]:
initial_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [20]:
initial_dataset = torchvision.datasets.ImageFolder(root=training_dataset_path, transform=initial_transform)
initial_loader = torch.utils.data.DataLoader(dataset=initial_dataset, batch_size=32, shuffle=False)


In [21]:
# Function to calculate mean and std
def get_mean_and_std(loader):
    mean = 0
    std = 0
    total_images_count = 0
    for images, _ in loader:
        images_count_in_a_batch = images.size(0)
        images = images.view(images_count_in_a_batch, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += images_count_in_a_batch
    mean /= total_images_count
    std /= total_images_count
    return mean, std

mean, std = get_mean_and_std(initial_loader)

In [22]:
# Updated Transformations with Data Augmentation
training_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

testing_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [23]:
# Datasets and Loaders
training_dataset = torchvision.datasets.ImageFolder(root=training_dataset_path, transform=training_transform)
testing_dataset = torchvision.datasets.ImageFolder(root=test_dataset_path, transform=testing_transform)

train_loader = torch.utils.data.DataLoader(dataset=training_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testing_dataset, batch_size=32, shuffle=False)


In [24]:
# Function to set device
def set_device():
    return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("CUDA available: ", torch.cuda.is_available())


CUDA available:  False


In [25]:

# Training Function
def train_nn(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs):
    device = set_device()
    model.to(device)
    
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1}/{n_epochs}")
        model.train()
        running_loss = 0.0
        running_correct = 0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (predicted == labels).sum().item()
        
        epoch_loss = running_loss*100 / len(train_loader)
        epoch_acc = running_correct *100 / total
        print(f"Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")
        
        test_acc = evaluate_model_on_test_set(model, test_loader)
        print(f"Test Accuracy: {test_acc:.4f}")
        
        scheduler.step()
    
    print("Training finished")
    return model


In [26]:

# Evaluation Function
def evaluate_model_on_test_set(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    device = set_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    accuracy = predicted_correctly_on_epoch / total
    return accuracy*100

# Model, Loss, Optimizer, and Scheduler
resnet18_model = torchvision.models.resnet18(pretrained=True)
num_features = resnet18_model.fc.in_features
number_of_classes = 10  # Adjust this based on your number of classes
resnet18_model.fc = nn.Linear(num_features, number_of_classes)

device = set_device()
resnet18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18_model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
print("done")

done


In [27]:
trained_model = train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, scheduler, n_epochs=20)


Epoch 1/20
Training Loss: 80.0476, Training Accuracy: 74.3847
Test Accuracy: 73.8971
Epoch 2/20
Training Loss: 45.3732, Training Accuracy: 85.4148
Test Accuracy: 73.1618
Epoch 3/20
Training Loss: 33.9285, Training Accuracy: 88.6964
Test Accuracy: 90.0735
Epoch 4/20
Training Loss: 20.6685, Training Accuracy: 94.2571
Test Accuracy: 87.1324
Epoch 5/20
Training Loss: 25.7016, Training Accuracy: 91.7046
Test Accuracy: 80.8824
Epoch 6/20
Training Loss: 20.8731, Training Accuracy: 94.1659
Test Accuracy: 90.8088
Epoch 7/20
Training Loss: 16.5918, Training Accuracy: 95.0775
Test Accuracy: 86.3971
Epoch 8/20
Training Loss: 8.0925, Training Accuracy: 97.5387
Test Accuracy: 95.5882
Epoch 9/20
Training Loss: 5.8156, Training Accuracy: 98.4503
Test Accuracy: 95.9559
Epoch 10/20
Training Loss: 2.4374, Training Accuracy: 99.4531
Test Accuracy: 96.6912
Epoch 11/20
Training Loss: 2.1992, Training Accuracy: 99.7265
Test Accuracy: 96.6912
Epoch 12/20
Training Loss: 2.5488, Training Accuracy: 99.3619
Test 